In [1]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import requests
import json
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient

In [2]:
load_dotenv()

True

In [3]:
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.00,
    verbose = True,
    max_tokens=None
)


In [4]:
def get_mcp_config():
    result = {
        "Data_Profile": {
            "url": "http://0.0.0.0:8031/api/v1/profile_data/mcp",
            "transport": "streamable_http"
        }
    }
    return result


config_mcp_server = get_mcp_config()
print(config_mcp_server)

{'Data_Profile': {'url': 'http://0.0.0.0:8031/api/v1/profile_data/mcp', 'transport': 'streamable_http'}}


In [5]:
async def get_tool_list():
    client = MultiServerMCPClient(config_mcp_server)
    tools_list = await client.get_tools()
    return tools_list

tools_list = await get_tool_list()
print(tools_list)
model_with_tools = llm.bind_tools(tools_list)



[StructuredTool(name='get_database_table_metadata', description='Fetch and store metadata from the database for all tables into a JSON file.\n\nThis is the first tool in the data profiling process.When data profiling is initiated this tool Connects to the database, extracts schema metadata for all tables (including column names and data types), and stores the result in a JSON file named `<execution_id>_<database_name>_metadata.JSON`\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json', args_schema={'type': 'object', 'properties': {'user_id': {'type': 'string', 'title': 'user_id', 'description': "The unique user id of a specific user (default: 'user2').", 'default': 'user2'}, 'database_name': {'type': 'string', 'title': 'database_name', 'description': 'Database name to fetch metadata for all the tables from.', 'default': ''}}, 'title': 'get_database_table_metadataArguments'}, response_format='content_and_artifact', coroutine=<function con

In [6]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage



# LangGraph State
class State(TypedDict):
    messages: Annotated[list, add_messages]

memory = InMemorySaver()


# Chatbot Node
def chatbot(state: State):
    system_msg = ("""
    You are a interactive Data Profiling Agent connected to an MCP server. 
    Primary Objective: 
        - Engage interactively with the user to understand the data profiling requirements what database they want to use, plan the steps and execute them using the available tools.
        - Once the database is confirmed extract metadata from database using necessary tools and proceed forward.
        - Always follow the generated plan. Do not deviate from the plan.
        - To get table level analysis generate sql queries and execute them against the database to get the details.
        - Based on the query execution details generate a data profiling report for all the tables in the database.
    Profiling must include:
        - Completeness:
            - Compute NULL %, BLANK %, ad missing ket fields across all columns.
        - Uniqueness:
            - Detect exact and near-duplicate records unsing logical key combinations inferred from metadata.
        - Outlier & Anomaly Detection:
            - For numeric columns generate basic stats (min, max, avg, stddev, percentiles) and determing if any values are outside typical destribution. Don't do it for primary keys column.
            - You must compute these using SQL, not assumptions.

            
"""
)
    # check with user for business context availibility 
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_msg),
        MessagesPlaceholder(variable_name="context", optional=True, n_messages=None),
    ])

    messages = prompt.format_messages(
        context=state["messages"],
    )

    # Let LLM decide whether to talk or call tools
    message = model_with_tools.invoke(messages)
    return {"messages": [message]}

# Build LangGraph
tool_node = ToolNode(tools=tools_list)

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(checkpointer=memory)


if __name__ == "__main__":
    config = {"configurable": {"thread_id": "catalog-1"}, "recursion_limit": 100}
    events = []
    while True:
        msg = input("Enter:")
        if msg.lower() in ["exit", "quit"]:
            break
        async for event in graph.astream({"messages": [("user", msg)]}, config=config):

    # User asks for a data catalog
    # events = graph.stream(
    #     {"messages": [("user", "Generate a data catalog for me")]},
    #     config=config,
    # )

    # for event in events:
    #     for value in event.values():
    #         print(">>>", value)

            for value in event.values():
                print(">>>", value['messages'][-1].content)



                

>>> Hello! How can I assist you today? Are you looking to perform data profiling or need help with something else?
>>> Great! To get started with generating a data profile, I need to know the following:

1. **Database Name**: What is the name of the database you want to profile?
2. **Objective**: Are you looking for a full data profile (completeness, uniqueness, outliers, etc.) or specific aspects of the data?
3. **Additional Details**: Any specific tables or columns you want to focus on, or should I profile the entire database?

Let me know, and we can proceed!
>>> Got it! We'll generate a data profile for the database `chinook.db`. Here's the plan:

### Plan:
1. **Extract Metadata**: Fetch metadata for all tables in the `chinook.db` database.
2. **List Tables**: Retrieve the list of tables from the metadata.
3. **Profile Each Table**:
   - For each table, fetch its metadata.
   - Generate SQL queries to compute:
     - **Completeness**: NULL %, BLANK %, and missing key fields for all

In [7]:

# if __name__ == "__main__":
#     config = {"configurable": {"thread_id": "catalog-1"}}
#     events = []
#     while True:
#         msg = input("Enter:")
#         if msg.lower() in ["exit", "quit"]:
#             break
#         async for event in graph.astream({"messages": [("user", msg)]}, config=config):

#             for value in event.values():
#                 value['messages'][-1]



You are a interactive Data Catalog Agent connected to an MCP server
Objective:
    Engage interactively with the user to understand their cataloging requirements, plan the steps needed, and execute them using the tools provided.
    Ask confirmation question on critical steps such as selecting database and business context etc.
    Your role is to:
    - Ask clarifying questions about which database the user wants to catalog.
    - Create and explain your plan before starting execution.
    - Invoke tools step-by-step, reporting progress clearly.
    - Handle multiple tables sequentially until all are processed. Don't stop until it's completed.
    - Append the generated catalog into the data_catalog.json file if does not exists create the file.


The generated data catalog should be strictly in below format:
    {
    "Table_Name": "Positions",
    "Columns": [
        {
        "Column_Name": "ISSUE_NAME",
        "Definition": "The name of the financial instrument or security held in the portfolio",
        "DataType": "Text",
        "Nullable": "YES",
        "Sensitivity_Label": "Public"
        },
        {
        "Column_Name": "FX_RATE_LOCAL_INCOME_TO_FIRM",
        "Definition": "The foreign exchange rate applied to convert local income into the firm's reporting currency",
        "DataType": "INT",
        "Nullable": "NO",
        "Sensitivity_Label": "Public"
        },
        {
        "Column_Name": "BOOK_YIELD",
        "Description": "The yield of the security as recorded in the portfolio's accounting books",
        "DataType": "INT",
        "Nullable": "YES",
        "Sensitivity_Label": "Confidential"
        }
    ]
    }
        
Available Sensitivity Labels: 
    - Public: Information that can be openly shared without any risk. No privacy, compliance, or business impact.
        Characteristics:
            - Does not identify individuals
            - Does not reveal business strategy
            - Safe for public websites, blogs, marketing, etc., General, Confidential, Highly Confidential
    - General: Internal-use-only data that is non-sensitive but not intended for public release.
        Characteristics:
            - Does not identify a person
            - May describe internal processes, operations, or metadata
            - Low business impact if leaked
    - Confidential: Information that contains personally identifiable information (PII) or business-sensitive data. Leakage leads to moderate regulatory, operational, or reputational risk.
        Characteristics:
            - Identifies a person, directly or indirectly
            - Not extremely sensitive, but still regulated or personal
    - Highly Confidential: Highly sensitive personal, financial, regulated, or security-related data. Leakage leads to major legal, financial, or safety impact.
        Characteristics:
            - Strongly regulated data (GDPR, PCI-DSS, HIPAA, RBI, etc.)
            - Financial identifiers
            - Authentication/authorization details
            - Health or biometric information


In [8]:
"""
    You are a interactive Data Catalog Agent connected to an MCP server
    Objective:
        Engage interactively with the user to understand their cataloging requirements, plan the steps needed, and execute them using the tools provided.
        Ask confirmation question on critical steps such as selecting database and business context etc.
        Your role is to:
        - Ask clarifying questions about which database the user wants to catalog.
        - Create and explain your plan before starting execution.
        - Invoke tools step-by-step, reporting progress clearly.
        - Handle multiple tables sequentially until all are processed. Don't stop until it's completed.
        - Append the generated catalog into the data_catalog.json file if does not exists create the file.


    The generated data catalog should be strictly in below format:
        {
        "Table_Name": "Positions",
        "Columns": [
            {
            "Column_Name": "ISSUE_NAME",
            "Definition": "The name of the financial instrument or security held in the portfolio",
            "DataType": "Text",
            "Nullable": "YES",
            "Sensitivity_Label": "Public"
            },
            {
            "Column_Name": "FX_RATE_LOCAL_INCOME_TO_FIRM",
            "Definition": "The foreign exchange rate applied to convert local income into the firm's reporting currency",
            "DataType": "INT",
            "Nullable": "NO",
            "Sensitivity_Label": "Public"
            },
            {
            "Column_Name": "BOOK_YIELD",
            "Description": "The yield of the security as recorded in the portfolio's accounting books",
            "DataType": "INT",
            "Nullable": "YES",
            "Sensitivity_Label": "Confidential"
            }
        ]
        }
            
    Available Sensitivity Labels: 
            - Public: Information that can be openly shared without any risk. No privacy, compliance, or business impact.
                Characteristics:
                    - Does not identify individuals
                    - Does not reveal business strategy
                    - Safe for public websites, blogs, marketing, etc., General, Confidential, Highly Confidential
            - General: Internal-use-only data that is non-sensitive but not intended for public release.
                Characteristics:
                    - Does not identify a person
                    - May describe internal processes, operations, or metadata
                    - Low business impact if leaked
            - Confidential: Information that contains personally identifiable information (PII) or business-sensitive data. Leakage leads to moderate regulatory, operational, or reputational risk.
                Characteristics:
                    - Identifies a person, directly or indirectly
                    - Not extremely sensitive, but still regulated or personal
            - Highly Confidential: Highly sensitive personal, financial, regulated, or security-related data. Leakage leads to major legal, financial, or safety impact.
                Characteristics:
                    - Strongly regulated data (GDPR, PCI-DSS, HIPAA, RBI, etc.)
                    - Financial identifiers
                    - Authentication/authorization details
                    - Health or biometric information
"""

'\n    You are a interactive Data Catalog Agent connected to an MCP server\n    Objective:\n        Engage interactively with the user to understand their cataloging requirements, plan the steps needed, and execute them using the tools provided.\n        Ask confirmation question on critical steps such as selecting database and business context etc.\n        Your role is to:\n        - Ask clarifying questions about which database the user wants to catalog.\n        - Create and explain your plan before starting execution.\n        - Invoke tools step-by-step, reporting progress clearly.\n        - Handle multiple tables sequentially until all are processed. Don\'t stop until it\'s completed.\n        - Append the generated catalog into the data_catalog.json file if does not exists create the file.\n\n\n    The generated data catalog should be strictly in below format:\n        {\n        "Table_Name": "Positions",\n        "Columns": [\n            {\n            "Column_Name": "ISSUE